# Bounding Box Explorations

We want to find a good set of axes so that when we query the range tree using the bounding box we generated for each polytope, we minimize the number of extraneous polytopes fetched where the bounding boxes intersect but the polytopes themselves do not intersect.

There are a few trade-offs to consider
1) Fetching the vertices can lead to a rough approximation of the oriented minimum bounding box. However, computing the vertices of the full dimensional polytope is expensive.
2) Using multiple bounding boxes the area under the intersection. However, since we pay $O((\log n)^d + k)$ penalty in the dimensions for the range tree search doubling $d$ might not make sense if the number of intersections is not reduced.

## Setup and Utility Functions

In [1]:
%cd /home/giorgian/Documente/Cercetare/Fairness/nn_fairness

/home/giorgian/Documente/Cercetare/Fairness/nn_fairness


In [25]:
import pickle
from polytope import Polytope, extreme, reduce, cheby_ball
import numpy as np
import quad
from compute_volume import lawrence_integrate_polytope
import json
from prob import ProbabilityDensityComputer
import math
from scipy.spatial import ConvexHull, QhullError
import nn_fairness as nnf
from compute_volume import dd_extreme
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from rtree import index
from itertools import product
from scipy.optimize import minimize
import miniball

In [3]:
with open('polys_adults.pickle', 'rb') as handle:
    res = pickle.load(handle)

for k, v in res['lpi_instance'].items():
    for poly in v:
        poly.deserialize()

In [33]:
def get_poly(group, i):
    return res['lpi_instance'][group][i]

def poly_volume(p):
    vertices = poly_vertices(p)

    return ConvexHull(vertices).volume

def poly_vertices(p):
    A = p.get_constraints_csr().toarray()
    b = p.get_rhs()
    c = np.array(p._get_col_bounds())
    A = np.concatenate([A, -np.eye(A.shape[1]), np.eye(A.shape[1])], axis=0)
    b = np.concatenate([b, c[:, 0], c[:, 1]])
    poly = reduce(Polytope(A, b))
    vertices = dd_extreme(poly)[0]
    return vertices

def poly_center(p):
    A = p.get_constraints_csr().toarray()
    b = p.get_rhs()
    c = np.array(p._get_col_bounds())
    A = np.concatenate([A, -np.eye(A.shape[1]), np.eye(A.shape[1])], axis=0)
    b = np.concatenate([b, c[:, 0], c[:, 1]])
    poly = reduce(Polytope(A, b))
    return cheby_ball(poly)[-1]
    

def get_bounding_box(lpi, directions=None):
    if directions is None:
        n = lpi.get_constraints_csr().shape[1]
        directions = np.eye(n)
    bb = []
    for i, v in enumerate(directions):
        min_bound = (v @ lpi.minimize(v))
        max_bound = (v @ lpi.minimize(-v))
        bb.append((min_bound, max_bound))

    return np.array(bb)

def get_directions_min(lpi):
    n = lpi.get_constraints_csr().shape[1]
    def clean_directions(d):
        d = np.array(d)
        for i in range(d.shape[0]):
            d[i] = d[i] / np.linalg.norm(d[i])
            for j in range(i+1, d.shape[0]):
                d[j] -= (d[j] @ d[i])*d[i]
                
        return d
    
    def f(x):
        directions = clean_directions(x.reshape(n, n))
        
        return bb_volume(get_bounding_box(lpi, directions))
    
    x0 = np.random.uniform(size=(n, n)).reshape(-1)
    return clean_directions(minimize(f, x0).x.reshape(n, n))
    
def bb_intersection_volume(bb_0, dir_0, bb_1, dir_1):
    A = np.concatenate((-dir_0, dir_0, -dir_1, dir_1), axis=0)
    b = np.concatenate((-bb_0[:, 0], bb_0[:, 1], -bb_1[:, 0], bb_1[:, 1]), axis=0)
    return lawrence_integrate_polytope(reduce(Polytope(A, b)))
    
    #return ConvexHull(extreme(reduce(Polytope(A, b)))[0]).volume

def get_pca(lpi):
    v = poly_vertices(lpi)
    return PCA(n_components=v.shape[1]).fit(v).components_

def get_min_bounding_box(lpi):
    directions = get_pca(lpi)
    return get_bounding_box(lpi, directions)

def bb_volume(bb, dirs=None, n_dims=None):
    if dirs is None or n_dims == bb.shape[0]:
        return np.prod(np.abs(bb[:, 1] - bb[:, 0]))
    else:
        A_parts = []
        b_parts = []
        for i in range(0, dirs.shape[0], n_dims):
            A_sub = dirs[i:i+n_dims]
            b_sub = bb[i:i+n_dims]
            A_parts.append(-A_sub)
            A_parts.append(A_sub)
            b_parts.append(-b_sub[:, 0])
            b_parts.append(b_sub[:, 1])
        A_full = np.concatenate(A_parts, axis=0)
        b_full = np.concatenate(b_parts, axis=0)
        try:
            return ConvexHull(dd_extreme(reduce(Polytope(A_full, b_full)))[0]).volume
        except QhullError:
            return ConvexHull(dd_extreme(reduce(Polytope(A_full, b_full)))[0], qhull_options="QJ").volume

In [29]:
def benchmark(methods):
    bounding_boxes = {}
    components = {}
    print('[Computing Components]')
    for label, lpi_polys in res['lpi_instance'].items():
        bounding_boxes[label] = {}
        components[label] = {}
        for method, fun in methods.items():
            c = fun(lpi_polys)
            components[label][method] = c
            bounding_boxes[label][method] = [get_bounding_box(lpi, c) for lpi in lpi_polys]
    
    print('[Computing Benchmark Figures]')
    stats = {}
    labels = list(res['lpi_instance'].keys())
    for l0, l1 in product(labels, labels):
        stats[(l0, l1)] = {}
        for m in methods.keys():

            b1 = bounding_boxes[l1][m]
            p = index.Property(dimension=b1[0].shape[0])
            box_index = index.Index(properties=p, interleaved=False)
            l1_volumes = []
            for i, b in enumerate(b1):
                box_index.insert(i, b.flatten())
                l1_volumes.append(bb_volume(b))

            
            c = components[l1][m]
            l0_volumes = []
            n = 0
            for lpi in tqdm(res['lpi_instance'][l0]):
                b0 = get_bounding_box(lpi, c)
                l0_volumes.append(bb_volume(b0, c, n_dims=lpi.get_constraints_csr().shape[1]))
                n += len(list(box_index.intersection(b0.flatten())))
                
            stats[(l0, l1)][m]= {
                'intersections': (n, len(res['lpi_instance'][l0]), len(res['lpi_instance'][l1])),
                f'{l0} BB volume': np.mean(l0_volumes),
                f'{l1} BB volume' : np.mean(l1_volumes)
            }
    return stats, bounding_boxes, components

## Methods to Benchmark

In [30]:
# Take all the vertices, find PCA for each vertex, find PCA of PCA
def get_directions_PCA(lpi_polys):
    components = np.concatenate([get_pca(lpi) for lpi in tqdm(lpi_polys)], axis=0)
    best_components = PCA(n_components=components.shape[1]).fit(components).components_
    return best_components

# Take all vertices and find PCA
def get_directions_PCA_1(lpi_polys):
    vertices = [poly_vertices(lpi) for lpi in tqdm(lpi_polys)]
    rescaled = []
    for vset in vertices:
        rescaled.append(vset - np.mean(vset, axis=0)[None, :])
    rescaled = np.concatenate(rescaled, axis=0)
    return PCA(n_components=rescaled.shape[-1]).fit(rescaled).components_


## Axis aligned
def get_directions_AA(lpi_polys):
    n_components = lpi_polys[0].get_constraints_csr().shape[1]
    return np.eye(n_components)

## Intersection of two bounding box: one axis aligned and one rotated at 45 degrees
def get_directions_double(lpi_polys):
    n_components = lpi_polys[0].get_constraints_csr().shape[1]
    dirs = np.eye(n_components)
    new_dirs = np.array(dirs)
    for i, j in zip(range(dirs.shape[0]), range(1, dirs.shape[0])):
        i %= dirs.shape[0]
        j %= dirs.shape[0]
        new_dirs[i] = (dirs[i] + dirs[j]) / np.sqrt(2)
        new_dirs[j] = (dirs[i] - dirs[j]) / np.sqrt(2)
    return np.concatenate([dirs, new_dirs], axis=0)
    
# Use furthest point from chebyshev center to define a set of vertices: use PCA of all directions
def get_directions_cheby(lpi_polys):
    components = np.concatenate([get_directions_cheby_polytope(lpi) for lpi in tqdm(lpi_polys)], axis=0)
    best_components = PCA(n_components=components.shape[1]).fit(components).components_
    return best_components


def get_directions_cheby_polytope(p):
    v = poly_vertices(p)
    c = poly_center(p)
    n = p.get_constraints_csr().shape[1]
    directions = []
    vp = np.array(v)
    for i in range(n):
        distances = np.linalg.norm(v - c, axis=-1)
        furthest = np.argmax(distances)
        d = (v[furthest] - c)/np.linalg.norm(v[furthest] - c)
        v = v - ((v - c) @ d)[:, None] * d
        directions.append(d)
    return np.stack(directions, axis=0)

def get_directions_mean(lpi_polys):
    components = np.concatenate([get_directions_mean_polytope(lpi) for lpi in tqdm(lpi_polys)], axis=0)
    best_components = PCA(n_components=components.shape[1]).fit(components).components_


def get_directions_mean_polytope(p):
    v = poly_vertices(p)
    c = np.mean(v, axis=0)
    n = p.get_constraints_csr().shape[1]
    directions = []
    vp = np.array(v)
    for i in range(n):
        distances = np.linalg.norm(v - c, axis=-1)
        furthest = np.argmax(distances)
        d = (v[furthest] - c)/np.linalg.norm(v[furthest] - c)
        v = v - ((v - c) @ d)[:, None] * d
        directions.append(d)
    return np.stack(directions, axis=0)

## Specify the benchmark and run it

In [31]:
methods = {
    'identity' : get_directions_AA,
    'double' : get_directions_double,
    'PCA' : get_directions_PCA,
    'PCA_1' : get_directions_PCA_1,
    'mean' : get_directions_mean,
    'cheby' : get_directions_cheby
}

In [34]:
stats, bounding_boxes, components = benchmark(methods)

[Computing Components]


  0%|          | 0/775 [00:00<?, ?it/s]

  0%|          | 0/775 [00:00<?, ?it/s]

  0%|          | 0/775 [00:00<?, ?it/s]

  0%|          | 0/775 [00:00<?, ?it/s]

...

  0%|          | 0/931 [00:00<?, ?it/s]

  0%|          | 0/931 [00:00<?, ?it/s]

  0%|          | 0/931 [00:00<?, ?it/s]

  0%|          | 0/931 [00:00<?, ?it/s]

  0%|          | 0/931 [00:00<?, ?it/s]

In [35]:
stats

{('white male',
  'white male'): {'identity': {'intersections': (209031, 775, 775),
   'white male BB volume': 0.02947638255211447}, 'double': {'intersections': (176989,
    775,
    775),
   'white male BB volume': 0.022996426706208373}, 'PCA': {'intersections': (333895,
    775,
    775),
   'white male BB volume': 0.3444643628866343}, 'PCA_1': {'intersections': (153760,
    775,
    775),
   'white male BB volume': 0.08752088524172408}, 'mean': {'intersections': (209032,
    775,
    775),
   'white male BB volume': 0.02947638255211447}, 'cheby': {'intersections': (350526,
    775,
    775),
   'white male BB volume': 0.566466208302958}},
 ('white male',
  'black male'): {'identity': {'intersections': (172210, 775, 931),
   'white male BB volume': 0.02947638255211449,
   'black male BB volume': 0.02561782188811023}, 'double': {'intersections': (124811,
    775,
    931),
   'white male BB volume': 0.01853321062287578,
   'black male BB volume': 0.02351765763826616}, 'PCA': {'interse